# Airtravel Optimization Program

In [1]:
import os
import pandas as pd
import csv
from itertools import permutations 
from math import pi,sin,cos,acos,floor

## Preparing

In [2]:
def read_file(file_name):
    """Read data in the file and Create Matrix"""
    datamatrix = []
    datafile = open(file_name)
    for line in datafile:
        datamatrix.append(line.split(","))
    datafile.close()
    return datamatrix

In [3]:
class Aircraft: # Class that contains all aircraft
    
    def __init__(self, model, manufacturer, max_capacity, current_capacity = 0):
        self._model = model # model of the aircraft
        self.manufacturer = manufacturer # manufacturer of the aircraft
        self._max_capacity = max_capacity # max capacity of the aircraft
        self._current_capacity = current_capacity # current capacity of the aircraft
    
    def get_max_capacity(self):
        return self._max_capacity # returns the max distance
    
    def get_maunfacturer(self):
        return self._manufacturer # returns the manufacturer
    
    def get_model(self):
        return self._model # returns the model

    def current_capacity(self):
        return self._current_capacity # returns the current capacity left in the plane
    
    def current_capacity(self, amount):
        if self._max_capacity == self._current_capacity: # This is just a checker.
            return "Sorry, the plane has a full tank"
        elif self._current_capacity + amount > self._max_capacity:
            return "Sorry, you have filled up by too much"
        else:
            self._current_capacity = amount # sets the capacity of the plane

class Airport:
    
    def __init__(self, airport_code, longitude, latitude, exchangeRate):
        self.airport_code = airport_code # airport code of the airport
        self._longitude = longitude # longitude of the airport
        self._latitude = latitude # latitude of the airport
        self._exchange_rate = exchangeRate # currency used by the airport
        
    def get_longitude(self):
        return self._longitude # returns the longitude of the airport
    
    def get_latitude(self):
        return self._latitude # returns the latitude of the airport
    
    def get_exchange_rate(self):
        return self._exchange_rate # returns the currency used by the airport
    
    def set_exchange_rate(self, exchangeRate):
        self._exchange_rate = exchangeRate # sets the currency of the airport
        

## Building the Aircraft

Time complexity: O(n)

In [4]:
def buildAircraft():
    """Builds objects for each of the aircraft - with attributes model, manufacturer, and range.
    Returns a dictionary of this"""
    aircraftDict = {}
    with open('aircraft.csv', newline='', encoding="utf8") as airplane_file: # opens the csv file
        reader = csv.reader(airplane_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for airplane in reader: # iterates through the reader
            if airplane[2] == "imperial":
                airRange = int(airplane[4]) * 1.609
            else:
                airRange = airplane[4]
            aircraftDict[airplane[0]] = Aircraft(airplane[0], airplane[3], airRange)
    return aircraftDict
    
    

## Building Airport Objects

Time complexity: 2n^2 + 4n ----> O(n^2)

In [5]:
def optimumRoute(listx):
    """Takes in a list of the route to be analysed. The airports are searched for in the csv.
    Objects of that airport are then created - with attributes latitude, longitude, and the exchange rate to Euro.
    It returns a dictionary with airport_code as key and object as value""" 

    airport_list = [] # creates a new list

    with open('airport.csv', newline='', encoding="utf8") as airport_file: # opens the csv file
        reader = csv.reader(airport_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for airport in reader: # iterates through the reader
            if airport[4] in listx:
                airport_code = airport[4] # assigns variable
                country_name = airport[3] # assigns variable
                longitude = airport[6] # assigns variable
                latitude = airport[7] # assigns variable
                templist = [airport_code, country_name, longitude, latitude]
                airport_list.append(templist)       
    
    country_currency_list = [] # creates a new list

    with open('countrycurrency.csv', newline='', encoding="utf8") as countrycurrency_file: # opens the csv file
        reader = csv.reader(countrycurrency_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for country in reader: # iterates through the reader
            temp_list = [] # temp list created
            temp_list.append(country[0]) # appends value to list
            temp_list.append(country[14]) # appends value to list
            country_currency_list.append(temp_list) # appends temp list to the main list
    
    currency_list = [] # creates a new list

    with open('currencyrates.csv', newline='', encoding="utf8") as currencyrates_file: # opens the csv file
        reader = csv.reader(currencyrates_file) # reads the cotents to a variable
        next(reader, None) # returns none at the end of the file
        for currency in reader: # iterates through the reader
            temp_list = [] # temp list created
            temp_list.append(currency[1]) # appends value to list
            temp_list.append(currency[2]) # appends value to list
            currency_list.append(temp_list) # appends temp list to the main list
            
    
    final_list = []
    for i in country_currency_list:
        for x in currency_list:
            if i[1] == x[0]:
                templist = [i[0], x[1]]
                final_list.append(templist)

    x = 0
    i = 0
    while x < len(airport_list):
        while i < len(final_list):
            if airport_list[x][1] == final_list[i][0]:
                airport_list[x].extend(final_list[i])
                break
            i+=1
        x+=1
    #Make a dictionary with the Airport code as the key and the value being the airport object of that key
    finalAirports = {}
    for i in airport_list:
        finalAirports[i[0]] = Airport(i[0], i[2], i[3], i[5])
   
    return finalAirports  

## Check Plane Capability

Should go here!!

## Distance

### Calculating all Route Permutations

Time complexity: O(n)

In [6]:
def allPerms(listx):
    """ Creates permutations of all possible routes using the input list of airports and plane """
    w = listx[1]
    x = listx[2]
    y = listx[3]
    z = listx[4]
    a = listx[0]
    count = 0
    permlist = []
    newpermlist = []
    perm = permutations([w,x,y,z]) 
    for i in list(perm): 
        permlist.append(i)

    for perms in permlist:
        perms = [a] + list(perms) + [a]
        newpermlist.append(perms)

    return newpermlist

### Calculates distances between each airport pair

Time complexity: n^4

In [7]:
def distanceBetweenAirports(latitude1,longitude1,latitude2,longitude2):
    radius_earth = 6371  # km
    theta1 = longitude1 * (2 * pi) / 360
    theta2 = longitude2 * (2 * pi) / 360
    phi1 = (90 - latitude1) * (2 * pi) / 360
    phi2 = (90 - latitude2) * (2 * pi) / 360
    distance = acos( sin(phi1) * sin(phi2) * cos(abs(theta1 - theta2)) +  cos(phi1) * cos(phi2) ) * radius_earth
    return floor(distance)

In [8]:
def permutations_no(listx, airport_list):
    """Takes list of route and aircraft required. Also takes in dictionary of airport objects. Creates permutation
    list of airports. Using the objects passed, it finds the distance between each airport and saves the leg 
    and distance as a key and value in a dictionary"""
    w = listx[1]
    x = listx[2]
    y = listx[3]
    z = listx[4]
    a = listx[0]
    count = 0
    permlist = []
    perm = permutations([w,x,y,z]) 
    for i in list(perm): 
        permlist.append(i)

    airport_distances = {}
    
    for perms in permlist:
        perms = [a] + list(perms) + [a]
        for i in range(0, len(perms) - 1):
            for j in airport_list:
                if j == perms[i]:
                    airport1 = airport_list[perms[i]]
                    for k in airport_list:
                        if k == perms[i+1]:
                            airport2 = airport_list[perms[i+1]]
                    distance = distanceBetweenAirports(float(airport1.get_longitude()), float(airport1.get_latitude()), float(airport2.get_longitude()), float(airport2.get_latitude()))
                    if distance != 0:
                        airport_distances['_'.join([(airport1.airport_code),(airport2.airport_code)])]  = distance

                
    return airport_distances

## Costing

### Calculating the Cost of each leg

Time complexity: O(n^2)

In [9]:
def findLegCosts(leg_distance_dict, airport_object_dict):
    """Takes in leg dictionary and airport objects. For each leg, it takes the departure airport 
    and gets the local currency conversion rate. It then multiplies it by the distance to get the cost of each leg.
    Returns a dictionary with the cost of each leg."""
    costDict = {}
    myKey = ""
    for i in leg_distance_dict:
        myKey = i[:3]
        x = 0
        cost = 0
        for j in airport_object_dict:
            if myKey == j:
                cost = round(float(airport_object_dict[j]._exchange_rate) * float(leg_distance_dict[i]), 2)
                costDict[i] = cost
            x+=1
    return costDict    

### Calculating the Cost of each Rotue

Time complexity: O(n^2)

In [10]:
def findRouteCost(myList, costDict):
    """This returns the total cost of each route - using the cost of each leg. Returned in a dictionary
    with: Key: route(tuple) & Values: Total cost"""
    routeCostDict = {}
    cost = 0
    x = 0
    
    while x < len(myList):
        i = 0
        cost = 0
        while i < (len(myList[x]) - 1):
            myKey = str(myList[x][i]) + "_" + str(myList[x][i+1])
            cost += costDict[myKey]
            cost = round(cost,2)
            i+=1
        routeCostDict[tuple(myList[x])] = cost
        x+=1
    return routeCostDict

## Check Aircraft Capability

Time complexity: O(n^2 + n)

In [13]:
def checkAircraftAllowed(dictAirplane, distanceDict, input_list, routeDict ):
    """Checks that the aircraft being used can do the route. Returns the routes that are
    only possible with the aircraft"""
    planeToFly = input_list[5]
    planeRange = dictAirplane[planeToFly].get_max_capacity()
    print(planeRange)
    distanceDict_copy = distanceDict.copy()
    for j in distanceDict_copy:
        if distanceDict_copy[j] < int(planeRange):
            distanceDict.pop(j)
    finalRouteDict_copy = finalRouteDict.copy()
    for i in finalRouteDict_copy:
        toRemove = False
        for j in distanceDict:
            x = 0
            while x < len(i) - 1:
                if str(i[x] + "_" + i[x + 1]) == j:
                    toRemove = True
                x+=1
        if toRemove == True:
            finalRouteDict.pop(i)

## Analysis

In [22]:
print("Please enter the name of the file containing the routes: ")
file_name = input("> ")

while os.path.isfile(file_name) != True:
    print("I am sorry. I do not believe that file exists. Remember, it is case sensitive. Please try again.") # Fix the indentation here
    print("=" * 75)
    print("")       
    file_name = input("> ")
    
print(f"Opening {file_name}....")
datamatrix = read_file(file_name)

for i in datamatrix:
    inputList = i # sample list passed
    inputList[5] = inputList[5].rstrip()
    airport_objects_dict = optimumRoute(inputList) # creates the objects for each airport
    dictOfAircrafts = buildAircraft() # Creates aircraft objects
    
    # create all the possible routes
    all_routes_list = allPerms(inputList) 
    
    # Finds distances and costs of each leg
    dict_routes_distances = permutations_no(inputList, airport_objects_dict)
    leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)
    
    # Finds total route cost
    finalRouteDict = findRouteCost(all_routes_list, leg_costs)
    
    # Removes distances that the aircraft cannot do
    checkAircraftAllowed(dictOfAircrafts, dict_routes_distances, inputList, finalRouteDict)
    if (len(finalRouteDict)) == 0:
        print("This route is not possible with the " + inputList[5])
        break
    cheapestRoute = min(finalRouteDict, key=finalRouteDict.get)
    cost = finalRouteDict[cheapestRoute]
    print("The cheapest route is " + str(cheapestRoute) + " and its cost is: " + str(cost))

Please enter the name of the file containing the routes: 
> test.csv
Opening test.csv....
15607.3
This route is not possible with the 777


### Unit tests

- Passing a plane that doesnt exist
- Passing a plane that can't fly 
- Check to make sure that no two consecutive airports are the same
- airport that doesn't exist
- check that units match with assumptions
- Check that all exchange rates are there

## TODO

- rewrite permutations algorithm 
- justify use of python's built in stuff
- Do unit tests
- check to see if o(n^4) can be rewritten!!!!!!

## Number of data structures/algorithms: 6
 - List
 - Tuple
 - dictionary 
 - Pricing algorithm
 - Plane allowed to fly route algorithm
 - Leg and distance calculation algorithm